# Busecke et al. GRL EUC Preprocessing
Notebook to convert/subset the data needed for the paper from the GFDL archive.
> Note: Files are staged in `/work` to reconstruct files, either replicate my workflow to [stage files](https://github.com/jbusecke/guides/blob/master/gfdl_file_management.md) or replace `/work` with `/archive`, which will take substantially longer (files are likely stored on tape and need to be retrieved).

On which domain is the EUC_shape detection performed? And is that described in the text?

## TODO before resubmission:
- Publish xarrayutils with momtools or standalone mom_tools (then change import below)
- Some units are messed up (Johnson is still in cm/s)
- Check the budget units in mv eof and budget plots (is it per kg?)
- I added eofs package to env

In [1]:
# For the dev
%load_ext autoreload
%autoreload 2

import warnings
warnings.simplefilter("ignore")

In [2]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from os.path import join
from dask.diagnostics import ProgressBar
from xgcm.autogenerate import generate_grid_ds
from xgcm import Grid
from xarrayutils.xgcm_utils import xgcm_weighted_mean, dll_dist

# There is a version of this from xarray...
from xarrayutils import aggregate

# #!!! at the point of publication This needs to be packaged into a proper module, with version that can be included in the requirments
# also in `basic.py`
import sys 
sys.path.append('/home/Julius.Busecke/projects/mom_tools')
from mom_budgets import add_vertical_spacing, add_split_tendencies

In [3]:
from global_oxygen.GRL_paper import mom_recombine_budgets, eq_mean, lat_slice, depth_slice, extract_shape, \
    extract_EUC_stats, xr_extract_contour, calculate_momentum_budget

from global_oxygen.basic import load_obs_dict

In [4]:
outdir = '../data/interim/GRL_Paper_clean/'
if not os.path.exists(outdir):
    os.mkdir(outdir)
    
def save_GRL_data(ds, name):
    filename = join(outdir,name)
    print('Saving to %s' %filename)
    if os.path.exists(filename):
        os.remove(filename)
    ds.to_netcdf(filename)

In [5]:
models = ['CM21deg', 'CM26']
cmip = ['CMIP5_piControl', 'CMIP5_rcp85']
obs = ['Bianchi', 'Johnson']

In [6]:
# The step to convert all runs to zarr is missing
# from mom_read import open_mom5_CM_ESM

# Load all preprocessed data
There are a lot of steps that need to be documented for the publication but for now work from here.

1. Import all the processed datasets needed
2. Average them all together for the equatorial slices in one go

In [7]:
def prelim_roi(obj):
    """Rough region cut to avoid large files"""
    obj = obj.copy()
    x_roi = slice(-260, -60)
    y_roi = slice(-20, 20)
    z_roi = slice(0,1000)
    roi = {
        'xt_ocean': x_roi,
        'xu_ocean': x_roi,
        'yt_ocean': y_roi,
        'yu_ocean': y_roi,
        'st_ocean': z_roi,
        'sw_ocean': z_roi,
    }
    for k,v in roi.items():
        if k in obj.dims:
            obj = obj.sel(**{k:v})
        if 'run' in obj.dims:
            obj = obj.sel(run='control')
    return obj

In [8]:
# this takes forever due to a strange behaviour in `add_vertical_spacing`. I have raised an issue over at 
# xarray(https://github.com/pydata/xarray/issues/2867)
models = ['CM21deg', 'CM26']
kwargs_dict = dict()
data = dict()

for name in models:
    data[name] = xr.open_zarr('../data/processed/%s.zarr' %name)
    # Add the vertical spacing
    data[name] = add_vertical_spacing(data[name])
    data[name] = add_split_tendencies(data[name])
#         data[name] = mom_recombine_budgets(data[name]) #moved to plotting to save space?
    data[name] = prelim_roi(data[name])
for k in data.keys():
    kwargs_dict[k] = {'model_processing':True}
print(data.keys())

Vertical spacing for vertical vel cell is approximated!!! Use with caution
Spacing for `dxte` is approximated!!! Use with caution
Spacing for `dytn` is approximated!!! Use with caution
Vertical spacing for vertical vel cell is approximated!!! Use with caution
Spacing for `dxte` is approximated!!! Use with caution
Spacing for `dytn` is approximated!!! Use with caution
dict_keys(['CM21deg', 'CM26'])


In [9]:
# now for the observations
# Gridded Obs
obs_dict = load_obs_dict(fid_dict=['Bianchi']) #!!! This violates the reproducibility by having an absolute path
ref = data['CM21deg'].isel(time=0).squeeze()
obs = {k:v.load().interp_like(ref) for k,v in obs_dict.items()}

# add model coords...
for k in obs.keys():
    for var in ['dxt', 'dyt', 'dzt']:
        obs[k].coords[var] = ref[var]
        
# Velocity Obs
obs['Johnson'] = xr.open_dataset('../data/processed/johnson_obs.nc')
obs['tao'] = xr.open_dataset('../data/processed/tao_obs.nc')

for k in obs.keys():
    kwargs_dict[k] = {'model_processing':False}
    data[k] = obs[k]

In [10]:
# CMIP equatorial slices (!!! need to reinterpolate also away from the equator for now just add)
cmip_path = '../data/processed/'
for name in ['CMIP5_tropical_combined_piControl', 'CMIP5_tropical_combined_rcp85']:
    short_name = name.replace('_tropical_combined', '')
    ds_cmip = xr.open_dataset(cmip_path+'%s.nc' %name).squeeze()
    # cmip o2 values are given as [umol/m^3] (check) and thus need to be divided by ref density. I assume this is 1035 kg/m^3 (!!!check)
    ds_cmip['o2'] = ds_cmip['o2'] / 1035
    
    #Create grid metrics
    ds_full = generate_grid_ds(ds_cmip, {'X':'lon', 'Y':'lat'})
    grid = Grid(ds_full)
    dlonc = grid.diff(ds_full.lon_left, 'X', boundary='fill', fill_value=np.nan)
    dlatc = grid.diff(ds_full.lat_left, 'Y', boundary='fill', fill_value=np.nan)
    
    ds_cmip.coords['dxt'], ds_cmip.coords['dyt'] = dll_dist(dlonc, dlatc, ds_full.lon, ds_full.lat)
    # there is some jump, for now just cut it off (I can do this cleanly with the boundary dicont)
    ds_cmip = ds_cmip.isel(lon=slice(0,-1))

    kwargs_dict[short_name] = {'model_processing':False}
    data[short_name] = ds_cmip.rename({'uo':'u', 'lev':'st_ocean', 'lon':'xt_ocean', 'lat':'yt_ocean'})

In [11]:
# Dont keep all variables or the files for archival get bloated
o2_budget_vars = [a for a in data['CM21deg'].data_vars if (('o2_' in a) or ('_o2' in a)) and ( a not in [
    'o2_btf',
    'o2_eta_smooth',
    'o2_rivermix',
    'o2_stf',
    'o2_xland',
    'o2_xlandinsert',
    'o2_xflux_adv',
    'o2_yflux_adv',
    'o2_zflux_adv',
    'o2_xflux_adv_recon',
    'o2_yflux_adv_recon',
    'o2_zflux_adv_recon',
    'o2_advection_y_recon_tracer',
    'o2_advection_z_recon_tracer',
    'o2_advection_y_recon_vel',
    'o2_advection_z_recon_vel',
    'o2_xflux_submeso',
    'o2_yflux_submeso',
    'o2_zflux_submeso',
])] + ['jo2']
keep_vars = ['o2', 'u', 'v', 'pot_rho_0', 'dzt', 'NINO34'] + o2_budget_vars

def xr_keep(obj, varlist=keep_vars):
    obj = obj.copy()
    keep_vars = [a for a in varlist if a in obj.data_vars]
    return obj.get(keep_vars)

## Computing the equatorial average and depth/lat slices

In [16]:
# Equatorial slices (average from 1S-1N) - with full time variability retained...
from global_oxygen.GRL_paper import interp_all, eq_mean
for k, ds in data.items():
    ds = ds.copy()
    filename = '%s_eq_mean.nc' %k
    if k not in ['tao']:
        ds_eq = prelim_roi(eq_mean(xr_keep(ds), **kwargs_dict[k]))
        print('saving to %s' %filename)
        with ProgressBar():
            save_GRL_data(ds_eq, filename)
del ds_eq, k, ds, var

saving to CM21deg_eq_mean.nc
Saving to ../data/interim/GRL_Paper_clean/CM21deg_eq_mean.nc
[########################################] | 100% Completed | 56.9s
saving to CM26_eq_mean.nc
Saving to ../data/interim/GRL_Paper_clean/CM26_eq_mean.nc
[########################################] | 100% Completed | 21min 10.0s
saving to Bianchi_eq_mean.nc
Saving to ../data/interim/GRL_Paper_clean/Bianchi_eq_mean.nc
[########################################] | 100% Completed |  0.2s
saving to Johnson_eq_mean.nc
Saving to ../data/interim/GRL_Paper_clean/Johnson_eq_mean.nc
saving to CMIP5_piControl_eq_mean.nc
Saving to ../data/interim/GRL_Paper_clean/CMIP5_piControl_eq_mean.nc
saving to CMIP5_rcp85_eq_mean.nc
Saving to ../data/interim/GRL_Paper_clean/CMIP5_rcp85_eq_mean.nc


## Lat slices along Johnson observations

In [12]:
for kk, ds in data.items():
    if kk not in ['tao', 'MIMOC']: #MIMOC has neither o2 nor u, and tao lat slices are not very useful?
        print(kk)
        filename = '%s_lat_slice.nc' %kk
        ref = data['Johnson'].u
        ds = ds.copy()
        ds = xr_keep(ds, ['u', 'o2'])
        for dd in ['xt_ocean', 'xu_ocean']:
            if dd in list(ds.dims):
                ds = ds.chunk({dd:-1})

        ds_lat_slice = (lat_slice(prelim_roi(ds), ref, **kwargs_dict[kk])) #

        if 'time' in ds_lat_slice.dims:
            ds_lat_slice = ds_lat_slice.mean('time')
        if 'month' in ds_lat_slice.dims:
            ds_lat_slice = ds_lat_slice.mean('month')
        print(len(ds_lat_slice.xt_ocean))
        with ProgressBar():
            save_GRL_data(ds_lat_slice, filename)

CM21deg
10
Saving to ../data/interim/GRL_Paper_clean/CM21deg_lat_slice.nc
[########################################] | 100% Completed |  8.6s
CM26
10
Saving to ../data/interim/GRL_Paper_clean/CM26_lat_slice.nc
[########################################] | 100% Completed |  2min 30.6s
Bianchi
10
Saving to ../data/interim/GRL_Paper_clean/Bianchi_lat_slice.nc
[########################################] | 100% Completed |  0.4s
Johnson
10
Saving to ../data/interim/GRL_Paper_clean/Johnson_lat_slice.nc
[########################################] | 100% Completed |  0.1s
CMIP5_piControl
10
Saving to ../data/interim/GRL_Paper_clean/CMIP5_piControl_lat_slice.nc
[########################################] | 100% Completed |  7.9s
CMIP5_rcp85
10
Saving to ../data/interim/GRL_Paper_clean/CMIP5_rcp85_lat_slice.nc
[########################################] | 100% Completed | 24.5s


## Slices at constant depth

In [17]:
# Depth slices
for k, ds in data.items():
    filename = '%s_depth_slice.nc' %k
    if k in ['CM21deg', 'CM26', 'Bianchi']:
        ds_slice = depth_slice(xr_keep(ds, ['o2', 'u']), st_ocean=250, **kwargs_dict[k]) #Here I only need the o2
        # Bin averaging the high res model
        if 'CM26' in k:
            temp = xr.Dataset()
            for var in ds_slice.data_vars:
                with ProgressBar():
                    temp[var] = aggregate(ds_slice[var].chunk({'xt_ocean':100, 'yt_ocean':100}),
                                           [('xt_ocean', 20),('yt_ocean',20)])
                
            ds_slice = temp                
        print('saving depth slice to %s' %filename)
        with ProgressBar():
            save_GRL_data(ds_slice, filename) 
            del ds_slice, k, ds, filename

saving depth slice to CM21deg_depth_slice.nc
Saving to ../data/interim/GRL_Paper_clean/CM21deg_depth_slice.nc
[########################################] | 100% Completed |  9.4s
saving depth slice to CM26_depth_slice.nc
Saving to ../data/interim/GRL_Paper_clean/CM26_depth_slice.nc
[########################################] | 100% Completed |  1min 52.7s
saving depth slice to Bianchi_depth_slice.nc
Saving to ../data/interim/GRL_Paper_clean/Bianchi_depth_slice.nc
[########################################] | 100% Completed |  0.2s


## Extract the EUC shape parameters

In [18]:
# extract EUC stats
EUC_boundary = 0.2
def process_and_save_euc_shape(data):
    data = data.copy()
    for k,v in data.items():
        if 'u' in v.data_vars:
            if k not in ['tao']:
                    if 'xt_ocean' in v.u.dims:
                        v = v.rename({'xt_ocean':'xu_ocean', 'yt_ocean':'yu_ocean', 'dyt':'dyu'})
                    euc_shape = extract_EUC_stats(v, EUC_boundary)
            else:
                # tao has no lat dimesion, so it has to be processed differently
                temp = extract_shape(v.u, EUC_boundary)
                euc_shape = xr.Dataset({'core_value_z':temp.value,
                                                'core_pos_z':temp.position,
                                                'thickness':temp.extent,
                                                'upper':temp.inner,
                                                'lower':temp.outer,
                                                })
            euc_shape.attrs['EUC_boundary'] = EUC_boundary

            filename = '%s_euc_shape.nc' %k
            print('Saving to %s' %filename)
            save_GRL_data(euc_shape, filename)

process_and_save_euc_shape(data)

Saving to CM21deg_euc_shape.nc
Saving to ../data/interim/GRL_Paper_clean/CM21deg_euc_shape.nc
Saving to CM26_euc_shape.nc
Saving to ../data/interim/GRL_Paper_clean/CM26_euc_shape.nc
Saving to Johnson_euc_shape.nc
Saving to ../data/interim/GRL_Paper_clean/Johnson_euc_shape.nc
Something went wrong during interpolation
x [230.0, 235.0]
y [0.22620001, 0.2]
c 0.2
Something went wrong during interpolation
x [260.0, 265.0]
y [0.21700001, 0.2]
c 0.2
Something went wrong during interpolation
x [140.0, 145.0]
y [0.2, 0.24416131]
c 0.2
Saving to tao_euc_shape.nc
Saving to ../data/interim/GRL_Paper_clean/tao_euc_shape.nc
Saving to CMIP5_piControl_euc_shape.nc
Saving to ../data/interim/GRL_Paper_clean/CMIP5_piControl_euc_shape.nc
Saving to CMIP5_rcp85_euc_shape.nc
Saving to ../data/interim/GRL_Paper_clean/CMIP5_rcp85_euc_shape.nc


# Extract the omz shape parameter

In [19]:
names = models + cmip + obs

ds_eq = dict()
for k in names:
    ds_eq[k] = xr.open_dataset(join(outdir, '%s_eq_mean.nc' %k))
    
# Average the oxygen observations over month of the year.
ds_eq['Bianchi'] = ds_eq['Bianchi'].mean('month') 

tilt_threshold = 80 # in mumol
tilt_roi = dict(xt_ocean=slice(-230, -90), st_ocean=slice(0,500))

omz_shape = dict()
for k, v in ds_eq.items():
    print(k)
    if 'o2' in v.data_vars:
        omz_shape = xr_extract_contour(v.sel(**tilt_roi).o2.load(),
                                             tilt_threshold * 1e-6,
                                             'xt_ocean',
                                             'st_ocean',
                                             showplot=False).to_dataset(name='depth')
        filename = '%s_omz_shape.nc' %k
        save_GRL_data(omz_shape, filename)
del omz_shape, k, ds, filename

CM21deg
Saving to ../data/interim/GRL_Paper_clean/CM21deg_omz_shape.nc
CM26
Saving to ../data/interim/GRL_Paper_clean/CM26_omz_shape.nc
CMIP5_piControl
Saving to ../data/interim/GRL_Paper_clean/CMIP5_piControl_omz_shape.nc
CMIP5_rcp85
Saving to ../data/interim/GRL_Paper_clean/CMIP5_rcp85_omz_shape.nc
Bianchi
Saving to ../data/interim/GRL_Paper_clean/Bianchi_omz_shape.nc
Johnson


## Momentum budget and mean map slices

In [20]:
for k in models:
    # calculate momentum budget and cut to roi
    mom_budget = calculate_momentum_budget(data[k]).sel(yu_ocean=slice(-10, 10),
                                                        yt_ocean=slice(-10, 10),
                                                        st_ocean=slice(250, 350),
                                                        sw_ocean=slice(250, 350))
    grid = Grid(mom_budget)
    # average budget over depth.
    mom_budget = xgcm_weighted_mean(grid, mom_budget, 'Z', ['dzt', 'dzu'])
    # average in time
    mom_budget = mom_budget.mean('time')
    filename = '%s_hor_momentum_budget.nc' %k
    with ProgressBar():
        save_GRL_data(mom_budget, filename)
    del mom_budget, grid, filename

Saving to ../data/interim/GRL_Paper_clean/CM21deg_hor_momentum_budget.nc
[########################################] | 100% Completed |  9.7s
Saving to ../data/interim/GRL_Paper_clean/CM26_hor_momentum_budget.nc
[########################################] | 100% Completed |  3min 17.1s


# EOF Stuff

In [9]:
eof_roi = dict(st_ocean=slice(0,500), xt_ocean=slice(-235,-80))

eq_mean_split = {k:mom_recombine_budgets(xr.open_dataset(join(outdir,'%s_eq_mean.nc' %k))) for k in models}

No `eddy_combined` generated for o2. dummy added.


In [10]:
#!!!
# Did I mention that and how the seasons are removed from the signal?
from global_oxygen.GRL_paper import remove_seas_simple, eof_wrapper

eof_vars = ['o2', 'u', 'o2_advection', 'o2_vertical_diff', 'o2_advection_eddy_recon', 'o2_advection_recon_w_diff', 'o2_eddy_combined']

# single variable EOFs
eof_z = dict()
for k, ds in eq_mean_split.items():
    eof_z[k] = dict()
    # Remove seasonal cycle
    ds = (remove_seas_simple(ds)+ds.mean('time')).load()
    # Focus on our roi
    ds = ds.sel(**eof_roi)
    ds = ds.transpose('time','st_ocean', 'xt_ocean',) # This needs to be done in the eof wrapper
    for var in eof_vars:
        filename = 'single_eof_z_%s_%s.nc' %(var,k)
        eof_z[k][var] = eof_wrapper(ds, fields=[var], weight_fields=['dzt'], normalize=False)
        save_GRL_data(eof_z[k][var], filename)   

Saving to ../data/interim/GRL_Paper_clean/single_eof_z_o2_CM21deg.nc
Saving to ../data/interim/GRL_Paper_clean/single_eof_z_u_CM21deg.nc
Saving to ../data/interim/GRL_Paper_clean/single_eof_z_o2_advection_CM21deg.nc
Saving to ../data/interim/GRL_Paper_clean/single_eof_z_o2_vertical_diff_CM21deg.nc
Saving to ../data/interim/GRL_Paper_clean/single_eof_z_o2_advection_eddy_recon_CM21deg.nc
Saving to ../data/interim/GRL_Paper_clean/single_eof_z_o2_advection_recon_w_diff_CM21deg.nc
Saving to ../data/interim/GRL_Paper_clean/single_eof_z_o2_eddy_combined_CM21deg.nc
Saving to ../data/interim/GRL_Paper_clean/single_eof_z_o2_CM26.nc
Saving to ../data/interim/GRL_Paper_clean/single_eof_z_u_CM26.nc
Saving to ../data/interim/GRL_Paper_clean/single_eof_z_o2_advection_CM26.nc
Saving to ../data/interim/GRL_Paper_clean/single_eof_z_o2_vertical_diff_CM26.nc
Saving to ../data/interim/GRL_Paper_clean/single_eof_z_o2_advection_eddy_recon_CM26.nc
Saving to ../data/interim/GRL_Paper_clean/single_eof_z_o2_adve

In [11]:
# multivariate EOFs
for var2 in [e for e in eof_vars if e != 'o2']:
    for k, ds in eq_mean_split.items():
        filename = 'multi_eof_z_o2_%s_%s.nc' %(var2,k)
        # Remove seasonal cycle
        ds = (remove_seas_simple(ds)+ds.mean('time')).load()
        ds = ds.sel(**eof_roi)
        ds = ds.transpose('time','st_ocean', 'xt_ocean',) # This needs to be done in the eof wrapper
        mv_eof_z = eof_wrapper(ds, fields=['o2', var2], weight_fields=['dzt', 'dzt'], n=3)
        save_GRL_data(mv_eof_z, filename)

Saving to ../data/interim/GRL_Paper_clean/multi_eof_z_o2_u_CM21deg.nc
Saving to ../data/interim/GRL_Paper_clean/multi_eof_z_o2_u_CM26.nc
Saving to ../data/interim/GRL_Paper_clean/multi_eof_z_o2_o2_advection_CM21deg.nc
Saving to ../data/interim/GRL_Paper_clean/multi_eof_z_o2_o2_advection_CM26.nc
Saving to ../data/interim/GRL_Paper_clean/multi_eof_z_o2_o2_vertical_diff_CM21deg.nc
Saving to ../data/interim/GRL_Paper_clean/multi_eof_z_o2_o2_vertical_diff_CM26.nc
Saving to ../data/interim/GRL_Paper_clean/multi_eof_z_o2_o2_advection_eddy_recon_CM21deg.nc
Saving to ../data/interim/GRL_Paper_clean/multi_eof_z_o2_o2_advection_eddy_recon_CM26.nc
Saving to ../data/interim/GRL_Paper_clean/multi_eof_z_o2_o2_advection_recon_w_diff_CM21deg.nc
Saving to ../data/interim/GRL_Paper_clean/multi_eof_z_o2_o2_advection_recon_w_diff_CM26.nc
Saving to ../data/interim/GRL_Paper_clean/multi_eof_z_o2_o2_eddy_combined_CM21deg.nc
Saving to ../data/interim/GRL_Paper_clean/multi_eof_z_o2_o2_eddy_combined_CM26.nc
